<a href="https://colab.research.google.com/github/fryall/fofo/blob/main/old_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os,glob
import cv2
from tqdm._tqdm_notebook import tqdm_notebook as tqdm # import was correct
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense,MaxPooling2D,GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten, Dense, InputLayer

In [ ]:
!pip install split-folders

In [ ]:
import splitfolders
image_size=[224,224]
input_folder = '/content/drive/MyDrive/try1'
splitfolders.ratio(input_folder, output="myTest",
                   seed=42, ratio=(.7, .3 ),
                   group_prefix=None)

In [ ]:
import tensorflow as tf

# Define the paths to the train and validation directories
train_dir = 'myTest/train'
val_dir = 'myTest/val'

# Load the training data
train_path = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),  # same as your image_size
    batch_size=64,          # adjust this as needed
    label_mode='categorical'  # or 'binary' depending on your labels
)

# Load the validation data
test_path= tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(224, 224),  # same as your image_size
    batch_size=64,          # adjust this as needed
    label_mode='categorical'  # or 'binary' depending on your labels
)

In [ ]:
from keras.applications import VGG16
VGG=VGG16(input_shape=(224,224,3),include_top=False,weights='imagenet')

In [ ]:
VGG.trainable=False

In [ ]:
input_shape = (224, 224, 3)

In [ ]:
model=Sequential()
model.add(InputLayer(shape=input_shape))
model.add(VGG)
model.add(GlobalAveragePooling2D())
model.add(Dense(units=250,activation="relu"))
model.add(Dense(units=125,activation="relu"))
model.add(Dense(units=20,activation="softmax"))

In [ ]:
print(VGG.output_shape)

In [ ]:
model.compile(optimizer='adam',
              loss="categorical_crossentropy",
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.98):
      print("\nReached 98% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("vgg16_1.keras", monitor="val_accuracy", verbose=1,
                             save_best_only=True, save_weights_only=False)
# Removed 'early' parameter as it's not part of ModelCheckpoint

early_stopping = EarlyStopping(monitor="val_accuracy", min_delta=0, patience=40,
                               verbose=1, mode='auto')
# Created a separate EarlyStopping callback

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, verbose=1)


In [ ]:
num_training_images = len(train_path.file_paths)
num_validation_images = len(test_path.file_paths)
print(num_training_images)
print(num_validation_images)

In [ ]:
steps_per_epoch = num_training_images // 64
validation_steps = num_validation_images // 64
print(steps_per_epoch)
print(validation_steps)

In [ ]:
history = model.fit(x=train_path, steps_per_epoch=60, epochs=40,
                 validation_data=test_path, validation_steps=25,
                 callbacks=[checkpoint])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have a trained model 'model' and test dataset 'test_path'
# Extract all test images and labels into numpy arrays
X_test = []
y_test = []
for images, labels in test_path:
    X_test.append(images.numpy())
    y_test.append(labels.numpy())

# Concatenate the batches into a single numpy array
X_test = np.concatenate(X_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

# Predict labels for the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)

# Plot the confusion matrix
plt.figure(figsize=(10, 10))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='RdPu')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

# Calculate precision, recall, F1 score, and accuracy
report = classification_report(y_true_classes, y_pred_classes, output_dict=True)
accuracy = report['accuracy']
precision = report['macro avg']['precision']
recall = report['macro avg']['recall']
f1_score = report['macro avg']['f1-score']

# Display the calculated metrics
print(f'Validation Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1_score:.4f}')

In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

# Print the test accuracy
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))

In [ ]:
# Save the entire model to a HDF5 file
model.save("my_model.h5")

In [ ]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# Assuming 'train_path' is your training dataset
class_names = train_path.class_names

# Save the class names to a text file
with open('labels.txt', 'w') as f:
  for class_name in class_names:
    f.write(class_name + '\n')

In [ ]:
from google.colab import files

files.download('labels.txt')